In [ ]:
!pip install tensorflow==2.15.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation

In [ ]:
!pip install -q keras-ocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.3 MB/s eta 0:00:00


In [ ]:
import keras_ocr

In [ ]:
import pandas as pd
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import cv2
import numpy as np
iteration=0
# keras-ocr will automatically download pretrained weights for the detector and recognizer.
pipeline = keras_ocr.pipeline.Pipeline()

# Function to perform OCR using Keras OCR and handle errors
def extract_text_with_keras_ocr(url):
    try:
        # Initialize image variable to avoid uninitialized variable issues
        img = None

        global iteration  # Access the global iteration variable

        print(f"Iteration: {iteration}")
        iteration += 1
        print(f"Attempting to download image from: {url}")

        # Download the image
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        print("Image downloaded successfully.")

        # Open the image using PIL
        img = Image.open(BytesIO(response.content))
        print("Image opened successfully with PIL.")

        # Convert PIL Image to OpenCV format (NumPy array)
        img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        print("Image converted to OpenCV format.")

        # Use keras-ocr's tools to read the image in the correct format
        img_array = keras_ocr.tools.read(img_cv)
        print("Image converted to NumPy array using keras-ocr tools.")

        # Perform OCR
        prediction_groups = pipeline.recognize([img_array])
        print("OCR performed successfully.")

        # Extract text from prediction groups
        extracted_text = ' '.join([text for text, _ in prediction_groups[0]])
        print("Text extracted successfully.")

        return extracted_text

    except requests.exceptions.RequestException as e:
        return f"Error downloading image from {url}: {str(e)}"
    except UnidentifiedImageError as e:
        return f"Error: The content could not be identified as an image from {url}. {str(e)}"
    except Exception as e:
        # Provide detailed error message with context if img is not defined
        if img is None:
            return f"Error processing image: Image not loaded or defined properly for URL {url}. {str(e)}"
        else:
            # Debugging print statement to capture state
            print(f"Unexpected error after loading image: {e}")
            return f"Error processing image from {url}: {str(e)}"

# Load the CSV file
df = pd.read_csv('/content/ocr.csv')

# Apply the function to the 'image_link' column and create a new 'keras_ocr_text' column
df['keras_ocr_text'] = df['image_link'].apply(extract_text_with_keras_ocr)

# Save the DataFrame to a new CSV file
df.to_csv('/content/keras_ocr.csv', index=False)

print("OCR processing completed and results saved to 'keras_ocr.csv'.")


Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
Iteration: 0
Attempting to download image from: https://m.media-amazon.com/images/I/61+40BOJSQL.jpg
Image downloaded successfully.
Image opened successfully with PIL.
Image converted to OpenCV format.
Image converted to NumPy array using keras-ocr tools.
1/1 [==============================] - 3s 3s/step
OCR performed successfully.
Text extracted successfully.
Iteration: 1
Attempting to download image from: https://m.media-amazon.com/images/I/516pgePFERL.jpg
Image downloaded successfully.
Image opened successfully with PIL.
Image converted to OpenCV format.
Image converted to NumPy array using keras-ocr tools.
2/2 [==============================] - 4s 650ms/step
OCR performed successfully.
Text extracted successfully.
Iteration: 2
Attempting to download image from: https://m.media-amazon.com/images/I/618UHY-K+pL.jpg
Image downloaded successfully.
Image opened successfully with PIL.
Image converted